In [10]:
import re
import pandas as pd
from tqdm import tqdm
with open('big.txt','r') as fd:
    lines = fd.readlines()
    words = []
    for line in lines:
        words += re.findall('\w+',line.lower())

print(len(words))
vocab = list(set(words))
print( "Vocabluray of the dataset:",len(vocab))

1115585
Vocabluray of the dataset: 32198


In [6]:
# Finding the Probability Distribution
word_probability = {}

for word in tqdm(vocab):
    word_probability[word] = float(words.count(word)/len(words))

100%|██████████| 32198/32198 [13:21<00:00, 40.19it/s]


In [11]:
#Text Preprocessing
#splitting

def split(word):
    parts = []
    for i in range(len(word) + 1):
        parts += [(word[ : i], word[i : ])]
    return parts

In [12]:
def delete(word):

    output = []
    for l,r in split(word):
        output.append(l + r[1:])
    return output

delete('loave')

['oave', 'lave', 'love', 'loae', 'loav', 'loave']

In [13]:
#swapping

def swap(word):

    output = []
    for l,r in split(word):
        if (len(r) > 1):
            output.append(l + r[1] + r[0] + r[2:])
    return output

swap('lvoe')


['vloe', 'love', 'lveo']

In [14]:
#replace
def replace(word):

    characters = 'abcdefghijklmnopqrstuvwxyz'
    output = []

    for l,r in split(word):
        for char in characters:
            output.append(l + char +  r[1:])
    return output

len(replace('lave'))

130

In [15]:
#insert
def insert(word):

    characters = 'abcdefghijklmnopqrstuvwxyz'
    output = []

    for l,r in split(word):
        for char in characters:
            output.append(l + char + r)

    return output

len(insert('lve'))

104

In [16]:
#Finding the Prediction (Level - 1)
#.1) Combining Possible Words


def edit(word):
    return list(set(insert(word) + delete(word) + swap(word) + replace(word)))


NameError: name 'spell_check_edit_1' is not defined

In [17]:
#Predicting the Word
def spell_check_edit_1(word, count = 5):

    output = []
    suggested_words = edit(word)

    for wrd in suggested_words:
        if wrd in word_probability.keys():
            output.append([wrd, word_probability[wrd]])

    return list(pd.DataFrame(output, columns = ['word','prob']).sort_values(by = 'prob', ascending = False).head(count)['word'].values)

In [18]:
spell_check_edit_1('famili')

['family']

In [19]:
def spell_check_edit_2(word, count = 5):

    output = []
    suggested_words = edit(word)       # Level one Edit

    for e1 in edit(word):
        suggested_words += edit(e1)    # Second Level Edit

    suggested_words = list(set(suggested_words))

    for wrd in suggested_words:
        if wrd in word_probability.keys():
            output.append([wrd, word_probability[wrd]])
    return list(pd.DataFrame(output, columns = ['word','prob']).sort_values(by = 'prob', ascending = False).head(count)['word'].values)

spell_check_edit_2('fameli')

['family', 'namely', 'fame', 'camelia', 'camel']